In [1]:
import os

In [2]:
%pwd

'c:\\Users\\ranji\\My Projects\\Projects\\Kidney-Disease-Classification-Deep-Learning-MLOps-Pipeline\\research'

In [3]:
os.chdir("../")

In [4]:
%pwd

'c:\\Users\\ranji\\My Projects\\Projects\\Kidney-Disease-Classification-Deep-Learning-MLOps-Pipeline'

In [ ]:
from dataclasses import dataclass
from pathlib import Path


@dataclass(frozen=True)
class PrepareBaseModelConfig:
    root_dir: Path
    base_model_path: Path
    updated_base_model_path: Path
    params_image_size: list
    params_learning_rate: float
    params_include_top: bool
    params_classes: int

In [6]:
from src.KidneyDiseaseClassifier.constants import *
from src.KidneyDiseaseClassifier.utils.common import read_yaml, create_directories

In [7]:
class ConfigurationManager:
    def __init__(
        self,
        config_filepath = CONFIG_FILE_PATH,
        params_filepath = PARAMS_FILE_PATH):

        self.config = read_yaml(config_filepath)
        self.params = read_yaml(params_filepath)

        create_directories([self.config.artifacts_root])

    

    def get_prepare_base_model_config(self) -> PrepareBaseModelConfig:
        config = self.config.prepare_base_model
        
        create_directories([config.root_dir])
        params_image_size = list(map(lambda x: int(x) if not isinstance(x, int) else x, self.params.IMAGE_SIZE))
        
        prepare_base_model_config = PrepareBaseModelConfig(
            root_dir=Path(config.root_dir),
            base_model_path=Path(config.base_model_path),
            updated_base_model_path=Path(config.updated_base_model_path),
            params_image_size=params_image_size,
            params_learning_rate=float(self.params.LEARNING_RATE),
            params_include_top=bool(self.params.INCLUDE_TOP),
            params_classes=int(self.params.CLASSES)
        )

        return prepare_base_model_config

In [8]:
import os
import urllib.request as request
from zipfile import ZipFile
import tensorflow as tf

In [9]:
class PrepareBaseModel:
    def __init__(self, config: PrepareBaseModelConfig):
        self.config = config

    
    def get_base_model(self):
        self.model = tf.keras.applications.InceptionV3(
            input_shape=self.config.params_image_size,
            weights="imagenet",
            include_top=self.config.params_include_top
        )

        self.save_model(path=self.config.base_model_path, model=self.model)

    

    @staticmethod
    def _prepare_full_model(model, classes, freeze_all, freeze_till, learning_rate, input_shape):
    # Freeze base model layers
        model.trainable = True
        for layer in model.layers:
            layer.trainable = False
        for layer in model.layers[-100:]: 
            layer.trainable = True

        inputs = tf.keras.Input(shape=input_shape)
        x = model(inputs, training=False)
        x = tf.keras.layers.GlobalAveragePooling2D()(x)
        x = tf.keras.layers.Dense(512, activation='relu')(x)
        x = tf.keras.layers.BatchNormalization()(x)
        x = tf.keras.layers.Dropout(0.5)(x)
        x = tf.keras.layers.Dense(256, activation='relu')(x)
        x = tf.keras.layers.BatchNormalization()(x)
        x = tf.keras.layers.Dropout(0.5)(x)
        x = tf.keras.layers.Dense(128, activation='relu')(x)
        x = tf.keras.layers.BatchNormalization()(x)
        x = tf.keras.layers.Dropout(0.5)(x)
        outputs = tf.keras.layers.Dense(units=classes, activation='softmax')(x)

        full_model = tf.keras.models.Model(inputs=inputs, outputs=outputs)

        full_model.compile(
            optimizer=tf.keras.optimizers.Adam(learning_rate=learning_rate),
            loss=tf.keras.losses.CategoricalCrossentropy(),
            metrics=["accuracy"]
        )

        return full_model
    
    
    def update_base_model(self):
        self.full_model = self._prepare_full_model(
            model=self.model,
            classes=self.config.params_classes,
            freeze_all=False,
            freeze_till=None,
            learning_rate=self.config.params_learning_rate,
            input_shape=self.config.params_image_size
        )

        self.save_model(path=self.config.updated_base_model_path, model=self.full_model)

    
        
    @staticmethod
    def save_model(path: Path, model: tf.keras.Model):
        path = str(path)
        if not path.endswith(".keras"):
            path += ".keras"
        model.save(path, include_optimizer=False) # Save full model
        print(f"Model saved to {path}")



In [10]:
try:
    config = ConfigurationManager()
    prepare_base_model_config = config.get_prepare_base_model_config()
    prepare_base_model = PrepareBaseModel(config=prepare_base_model_config)
    prepare_base_model.get_base_model()
    prepare_base_model.update_base_model()
except Exception as e:
    raise e

[2025-07-12 14:55:05,304: INFO: common: yaml file: config\config.yaml loaded successfully]
[2025-07-12 14:55:05,306: INFO: common: yaml file: params.yaml loaded successfully]
[2025-07-12 14:55:05,307: INFO: common: created directory at: artifacts]
[2025-07-12 14:55:05,308: INFO: common: created directory at: artifacts/prepare_base_model]
[2025-07-12 14:55:06,997: WARNING: saving_utils: Compiled the loaded model, but the compiled metrics have yet to be built. `model.compile_metrics` will be empty until you train or evaluate the model.]
Model saved to artifacts\prepare_base_model\base_model.keras
Model saved to artifacts\prepare_base_model\base_model_updated.keras
